# Laboratorio de Introducción al Procesamiento de Lenguaje Natural

# Tarea 2

El objetivo de este laboratorio es realizar diferentes experimentos para representar y clasificar textos. Para esto se trabajará con un corpus para análisis de sentimiento, creado para la competencia [TASS 2020](http://www.sepln.org/workshops/tass/) (IberLEF - SEPLN).

### Entrega
Deberán entregar un archivo *.ipynb* con su solución, que incluya código, comentarios y respuestas a las preguntas que se incluyen al final de este notebook.

El plazo de entrega de la tarea 2 cierra el **20 de junio a las 23:59 horas**.

### Plataforma sugerida
Sugerimos que utilicen la plataforma [Google colab](https://colab.research.google.com/), que permite trabajar colaborativamente con un *notebook* de python. Al finalizar pueden descargar ese *notebook* en un archivo .ipynb, incluyendo las salidas ya ejecutadas, con la opción ```File -> Download -> Download .ipynb```.

### Aprobación del laboratorio
Para aprobar el laboratorio se exige como mínimo:
* Probar dos enfoques diferentes para la representación de tweets (uno basado en BoW y otro en word embeddings)
* Probar al menos dos modelos de aprendizaje automático con cada representación
* Comparar los resultados con los obtenidos por el modelo de pysentimiento.
El preprocesamiento, las pruebas con otras formas de representación de los tweets, los experimentos con otros modelos de aprendizaje automático, incluyendo aprendizaje profundo, entre otros posibles experimentos, no son requisito para aprobar el laboratorio, aunque aportan a la nota final.



**Grupo 02**
- Natalie Valentina Alaniz Ferreira, natalie.alaniz@fing.edu.uy
- Agustín Matías Martínez Acuña, agustin.martinez.acunaa@fing.edu.uy
- Matías Fernando Rama Perdomo, matias.rama@fing.edu.uy

## Parte 1 - Carga y preprocesamiento del corpus

Para trabajar en este notebook deben cargar los tres archivos disponbiles en eva: train.csv, devel.csv y test.csv.

La aplicación de una etapa de preprocesamiento similar a la implementada en la tarea 1 es opcional. Es interesante hacer experimentos con y sin la etapa de preprocesamiento, de modo de comparar resultados (sobre el corpus de desarrollo, devel.csv) y definir si se incluye o no en la solución final.



### Carga

In [4]:
# Carga de los datasets
import csv

# Carga del archivo train.csv
with open('datos/train.csv', newline='', encoding="utf-8") as corpus_csv:
    reader = csv.reader(corpus_csv)
    next(reader) # Saltea el cabezal del archivo
    train_set = [x for x in reader]

# Carga del archivo devel.csv
with open('datos/devel.csv', newline='', encoding="utf-8") as corpus_csv:
    reader = csv.reader(corpus_csv)
    next(reader) # Saltea el cabezal del archivo
    devel_set = [x for x in reader]

# Carga del archivo test.csv
with open('datos/test.csv', newline='', encoding="utf-8") as corpus_csv:
    reader = csv.reader(corpus_csv)
    next(reader) # Saltea el cabezal del archivo
    test_set = [x for x in reader]

# Carga del archivo stop_words_esp_anasent.csv
with open('datos/stop_words_esp_anasent.csv', newline='', encoding="utf-8") as corpus_csv:
    reader = csv.reader(corpus_csv)
    next(reader) # Saltea el cabezal del archivo
    stop_words_esp_anasent = [x[0] for x in reader]


### Preprocesamiento

In [5]:
# Algunos procedimientos y funciones auxiliares
def imprimir_tweet_polaridad(tweet):
  print("TWEET: "+ tweet[1])
  print("POLARIDAD: "+ tweet[2])

In [6]:
import re
import random

# Preprocesamiento de los tweets

# Definición de las sustituciones a realizar
expresiones = [[r'#\w+','HASHTAG'],
                  [r'http\S+', 'URL'], ['\?\?*', '?'],
                  [r'\!\!*', '!'],['@\w+', 'USUARIO'],
                  [r'\?', '? '],
                  ['!', '! '],
                  [',', ', '],
                  [' ,', ','],
                  [r'(?<!\.)\.(?!\.)', '. '],
                  [' q ', ' que '],
                  [' ke ', ' que '],
                  [' Q ', ' Que '],
                  [' d ', ' de '],
                  [r' (xq|pq) ', ' porque '],
                  [' XQ ', ' porque '],
                  [' m ', ' me '],
                  [' m ', ' me '],
                  ['M ', 'Me '],
                  [' x ', ' por '],
                  [r' (porfis|porfa|xfa) ', ' por favor '],
                  [r'\b(?:a*ja+j[aj]*)+\b', 'jaja'],
                  [r'\b(?:a*ha+h[ah]*)+\b', 'jaja'],
                  [r'(?i) ud(\. | )', ' Usted '],
                  [r'(?i) uds(\. | )', ' Ustedes '],
                  ['\.(\.)+', '...'],
                  [r'\s(\s)+', ' '],
                  [':', ': ']
                  ]

# Se aplican todas las sustituciones definidas en "expresiones"
def preprocesar_texto(texto):
    for norm in expresiones:
        texto = re.sub(str(norm[0]), str(norm[1]), texto)
    return texto

def preproc_dataset(dataset):
  res = []
  for twit in dataset:
    twit_preproc = preprocesar_texto(twit[1])
    res.append([twit[0], twit_preproc, twit[2]])
  return res

# Variables globales para los conjuntos preprocesados
train_set_preproc = preproc_dataset(train_set)
devel_set_preproc = preproc_dataset(devel_set)
test_set_preproc = preproc_dataset(test_set)

# Un ejemplo de train_set_preproc para corroborar que esté andando
imprimir_tweet_polaridad(random.choice(train_set_preproc))


TWEET: Sssh, creo que estoy viendo a los Reyes... Pero, que hacen? Llevan palanquetas y los sacos vacíos, espera! Están llevándose la cuberteria! 
POLARIDAD: NONE


## Parte 2 - Representación de los tweets

Para representar los tweets se pide que experimenten con modelos basados en Bag of Words (BoW) y con Word Embeddings.

Para los dos enfoques podrán elegir entre diferentes opciones:

**Bag of Words**

* BOW estándar: se recomienda trabajar con la clase [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) de sklearn, en particular, fit_transform y transform.
* BOW filtrando stop-words: tienen disponible en eva una lista de stop-words para el español, adaptada para análisis de sentimiento (no se filtran palabras relevantes para determinar la polaridad, como "no", "pero", etc.).
* BoW usando lemas: pueden usar herramientas de spacy.
* BOW seleccionando las features más relevantes: se recomienda usar la clase [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=select%20k%20best#sklearn.feature_selection.SelectKBest) y probar con diferentes valores de k (por ejemplo, 10, 50, 200, 1000).
* BOW combinado con TF-IDF: se recomienda usar la clase [TfidfVectorizer](https://https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

**Word Embeddings**

* A partir de los word embeddings, representar cada tweet como el vector promedio (mean vector) de los vectores de las palabras que lo componen.
* A partir de los word embeddings, representar cada tweet como la concatenación de los vectores de las palabras que lo componen (llevando el vector total a un largo fijo).

Se recomienda trabajar con alguna de las colecciones de word embeddings disponibles en https://github.com/dccuchile/spanish-word-embeddings. El repositorio incluye links a ejemplos y tutoriales.


Se pide que prueben al menos una opción basada en BoW y una basada en word embeddings.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.keyedvectors import KeyedVectors

### Representación BoW

In [15]:
def tweets_from_corpus(corpus):
     return [tuits[1] for tuits in corpus]

#### BoW Estándar

In [16]:
def representar_en_bow(frase, vectorizer):
    return vectorizer.transform([frase])

In [17]:
def bow_standard(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    return vectorizer, X.toarray()

#### BOW filtrando stop-words

In [18]:
def corpus_to_bow_stopwords(corpus):
    vectorizer = CountVectorizer(stop_words=stop_words_esp_anasent)
    X = vectorizer.fit_transform(corpus)
    return vectorizer , X.toarray()

#### BoW combinado con TF-IDF

### Representación Word Embedding

In [19]:
def corpus_to_bow_tf_idf(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return vectorizer, X.toarray()

In [20]:
# Representación de los tweets usando word embeddings
wordvectors_file_vec = 'embeddings-l-model.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

#### WE Mean Vector

In [21]:
# Representación de los tweets usando word embeddings representando cada tweet como el vector promedio (mean vector) de los vectores de las palabras que lo componen.
def mean_vector(tweet,sin_stop_words, pre_procesado, largo_vector):
    words_vectors = []
    for word in tweet.split():
        if sin_stop_words and word in stop_words_esp_anasent:
            continue

        if pre_procesado:
            word = preprocesar_texto(word)

        if word in wordvectors.key_to_index.keys():
            words_vectors.append(wordvectors[word])

    if len(words_vectors) == 0:
        mean_vector = np.zeros(largo_vector)
    else:
        mean_vector = np.mean(words_vectors, axis=0)[0:largo_vector]
    return mean_vector


#### WE Concatenación de Vectores

In [22]:
def representar_tuit_concatvec_we(tuit, tamanio_rep):
    palabras = tuit.split()
    rep = []
    for word in palabras:
        # Hay que ver que hacer con las palabras que no tienen representación, las estamos ignorando
        # pero se puede asignarle a las ignoradas un vector fijo por ej [0,0,0,...,0]
        if word in wordvectors:
            word_vector = wordvectors[word][:tamanio_rep]
        else:
            word_vector = np.zeros(tamanio_rep)
        rep.extend(word_vector)
    return rep

## Parte 3 - Clasificación de los tweets

Para la clasificación de los tweets es posible trabajar con dos enfoques diferentes:

* Aprendizaje Automático basado en atributos: se pide probar al menos dos modelos diferentes, por ejemplo, Multi Layer Perceptron ([MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)) y Support Vector Machines ([SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)), y usar al menos dos formas de representación de tweets (una basada en BoW y otra basada en word embeddings). Se publicó en eva un léxico de palabras positivas y negativas que puede ser utilizado para generar atributos.

* Aprendizaje Profundo: se recomienda experimentar con alguna red recurrente como LSTM. En este caso deben representar los tweets an base a word embeddings.

Deberán usar el corpus de desarrollo (devel.csv) para comparar resultados de diferentes experimentos, variando los valores de los hiperparámetros, la forma de representación de los tweets, el preprocesamiento, los modelos de AA, etc.

Tanto para la evaluación sobre desarrollo como para la evaluación final sobre test se usará la medida [Macro-F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html) (promedio de la medida F1 de cada clase).

In [63]:
# imports
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from pysentimiento import create_analyzer
import matplotlib.pyplot as plt

# Variables globales para los clasificadores
plt.style.use('fivethirtyeight')
sentimiento_map = {"NEU": "NONE", "NEG": "N", "POS": "P"}
tamanio = 100
cantidad_entradas_vec_tuit = 300
tamanio_vec_concat_we = 80

In [25]:
# Funciones auxiliares para generar los array de datos junto con sus polaridades para cada enfoque
def generar_datos_polaridad_concatvec_we(tamanio_rep_palabras, cantidad_palabras_tuit, corpus):
    polaridad = []
    datos = []

    for entrada in  corpus:
        vec_entrada = representar_tuit_concatvec_we(entrada[1].lower(),tamanio_rep_palabras)
        #Hago esto por si se pasa de largo
        vec_entrada = vec_entrada[0:min(len(vec_entrada), cantidad_palabras_tuit)]
        datos.append(vec_entrada)

        polaridad.append(entrada[2])

    datos_padded = [np.pad(vec, (0, cantidad_palabras_tuit - len(vec)), mode='constant') for vec in datos]

    datos = np.vstack(datos_padded)
    polaridad = np.array(polaridad)
    return datos, polaridad

def generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec, sin_stop_words, pre_procesado, corpus):
    polaridad = []
    datos = []

    for entrada in corpus:
        vec_entrada = mean_vector(entrada[1].lower(), sin_stop_words, pre_procesado, cantidad_entradas_vec)

        vec_entrada = np.array(vec_entrada)

        vec_entrada = vec_entrada[0:cantidad_entradas_vec]
        polaridad.append(entrada[2])
        datos.append(vec_entrada)

    polaridad = np.array(polaridad)
    return datos, polaridad

def generar_datos_polaridad_bow_stopwords(corpus, vectorizer):
    polaridad = []
    datos = []
    for entrada in corpus:
        vec_entrada = representar_en_bow(entrada[1].lower(), vectorizer)
        vec_entrada = vec_entrada.toarray()
        polaridad.append(entrada[2])
        datos.append(vec_entrada)

    datos = np.concatenate(datos, axis=0)
    polaridad = np.array(polaridad)
    return datos, polaridad

def generar_array_polaridad(corpus):
    return [x[2] for x in corpus]

#### Experimentos MLP

##### MLP utilizando BoW stopwords

In [26]:
def entrenar_mlp_bow_stopwords(corpus, vectorizer):
    datos, polaridad = generar_datos_polaridad_bow_stopwords(corpus, vectorizer)
    clf = MLPClassifier(random_state=1, max_iter=300)
    clf.fit(datos, polaridad)
    return clf

In [27]:
# Test MLP BoW Stopwords

def macroF1_bow(clf, vectorizer, corpus):
    datos, y_real = generar_datos_polaridad_bow_stopwords(corpus, vectorizer)
    y_pred = [clf.predict(representacion_tuit.reshape(1, -1)) for representacion_tuit in datos]

    return f1_score(y_real, y_pred, average="macro")

##### MLP utilizando Mean Vector (WE)

In [28]:
def entrenar_mlp_mean_vector(cantidad_entradas_vec, sin_stop_words, pre_procesado, corpus):
  clf = MLPClassifier(random_state=1, max_iter=30000)
  datos, polaridad = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus)
  clf.fit(datos, polaridad)

  return clf

### Experimentos SVM

#### SVM utilizando WE

##### SVM con concatenación de vectores

In [29]:
def entrenar_svm_concatvec(tamanio_rep_palabras, cantidad_palabras_tuit,  corpus):
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    datos, polaridad = generar_datos_polaridad_concatvec_we(tamanio_rep_palabras, cantidad_palabras_tuit, corpus)
    clf.fit(datos, polaridad)
    return clf

In [30]:
# Test SVM con concatvec
def predecir_polaridad_concat(tuit, cantidad_entradas_vec, clf):
    if cantidad_entradas_vec - len(tuit) < 0:
        tuit = tuit[0:cantidad_entradas_vec]
    else:
        tuit = np.pad(
            tuit,
            (0, cantidad_entradas_vec - len(tuit)),
            mode="constant",
        )
    return clf.predict([tuit])

def macro_F1_concatvec_we(tamanio_rep_palabras, cantidad_entradas_vec, clf, corpus):
    datos, y_real = generar_datos_polaridad_concatvec_we(tamanio_rep_palabras, cantidad_entradas_vec, corpus)
    y_pred = [predecir_polaridad_concat(x, cantidad_entradas_vec, clf) for x in datos]
    return f1_score(y_real, y_pred, average="macro")

##### SVM con Mean Vector

In [31]:
# SVM Mean Vector
def entrenar_svm_mean_vector(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus):
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    datos, polaridad = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus)
    clf.fit(datos, polaridad)
    return clf

In [77]:
def predecir_polaridad_mean_vector(tuit, cantidad_entradas_vec, clf):
    if cantidad_entradas_vec - len(tuit) < 0:
        representacion_tuit = tuit[0:cantidad_entradas_vec]
    else:
        representacion_tuit = np.pad(
           tuit,
            (0, cantidad_entradas_vec - len(tuit)),
            mode="constant",
        )
    return clf.predict([representacion_tuit])

def macro_F1_mean_vector(cantidad_entradas_vec,sin_stop_words, pre_procesado, clf, corpus):
    datos, y_real = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec, sin_stop_words, pre_procesado, corpus)
    y_pred = [predecir_polaridad_mean_vector(x, cantidad_entradas_vec, clf) for x in datos]
    return f1_score(y_real, y_pred, average="macro")

def accuracy_score_mean_vector(cantidad_entradas_vec,sin_stop_words, pre_procesado, clf, corpus):
    datos, y_real = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec, sin_stop_words, pre_procesado, corpus)
    y_pred = [predecir_polaridad_mean_vector(x, cantidad_entradas_vec, clf) for x in datos]
    return accuracy_score(y_real, y_pred)

### Ensamble Voting de SVM y MLP

In [33]:
# Ensamble Voting de SVM y MLP
def entrenar_ensamblado_de_votacion_svm_mlp(cantidad_entradas_vec, sin_stop_words, pre_procesado, corpus):
  clf_svm = entrenar_svm_mean_vector(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus)
  clf_mlp = entrenar_mlp_mean_vector(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus)
  datos, polaridad = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec,sin_stop_words, pre_procesado, corpus)

  clf_votacion = VotingClassifier(estimators=[('svm', clf_svm), ('mlp', clf_mlp)], voting='hard')
  clf_votacion.fit(datos, polaridad)

  return clf_votacion

### Resultados de los experimentos

In [101]:
# MLP con BoW Entrenamiento

vectorizer_bow_sw, x_sw = corpus_to_bow_stopwords(tweets_from_corpus(train_set))
clf_mlp_bow_stopwords = entrenar_mlp_bow_stopwords(train_set, vectorizer_bow_sw)

vectorizer_bow_tf_idf, x_tf_idf = corpus_to_bow_tf_idf(tweets_from_corpus(train_set))
clf_mlp_bow_tf_idf = entrenar_mlp_bow_stopwords(train_set, vectorizer_bow_tf_idf)

In [102]:
# MLP con BoW
res = []
headers = ['', 'Macro F1']

macroF1_bow_sw =  macroF1_bow(clf_mlp_bow_stopwords, vectorizer_bow_sw, devel_set)
res.append(["BoW sin SW", macroF1_bow_sw])

macroF1_bow_tf_idf = macroF1_bow(clf_mlp_bow_tf_idf, vectorizer_bow_tf_idf, devel_set)
res.append(["BoW TF-IDF", macroF1_bow_tf_idf])

print("MLP utilizando BoW")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

MLP utilizando BoW
╒════════════╤════════════╕
│            │   Macro F1 │
╞════════════╪════════════╡
│ BoW sin SW │    0.55153 │
├────────────┼────────────┤
│ BoW TF-IDF │    0.55004 │
╘════════════╧════════════╛


In [46]:
entrenamiento_agrandado = np.concatenate((np.array(train_set),np.array(devel_set)), axis=0)

In [97]:
# Entrenamiento MLP Mean Vector

clf_mlp_mean_vector_sin_stop_words_preproc = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit, True, True, train_set)
clf_mlp_mean_vector_sin_stop_words = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit ,True,False,train_set)
clf_mlp_mean_vector_preproc = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit,False,True, train_set)
clf_mlp_mean_vector = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit,False,False, train_set)

clf_mlp_mean_vector_sin_stop_words_preproc_agrandado = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit, True, True, entrenamiento_agrandado)
clf_mlp_mean_vector_sin_stop_words_agrandado = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit ,True,False,entrenamiento_agrandado)
clf_mlp_mean_vector_preproc_agrandado = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit,False,True, entrenamiento_agrandado)
clf_mlp_mean_vector_agrandado = entrenar_mlp_mean_vector(cantidad_entradas_vec_tuit,False,False, entrenamiento_agrandado)

In [63]:
# MLP con mean vector
res = []
headers = ['Sin SW', 'Pre-procesado', 'Macro F1']

macro_f1_mlp_mean_vector_sin_stop_words_preproc = macro_F1_mean_vector(cantidad_entradas_vec_tuit, True, True, clf_mlp_mean_vector_sin_stop_words_preproc, devel_set)
res.append(["Sí","Sí", macro_f1_mlp_mean_vector_sin_stop_words_preproc])

macro_f1_mlp_mean_vector_sin_stop_words = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_mlp_mean_vector_sin_stop_words, devel_set)
res.append(["Sí", "No", macro_f1_mlp_mean_vector_sin_stop_words])

macro_f1_mlp_mean_vector_pre_proc =  macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_mlp_mean_vector_preproc, devel_set)
res.append(["No", "Sí", macro_f1_mlp_mean_vector_pre_proc])

macro_f1_mlp_mean_vector = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_mlp_mean_vector, devel_set)
res.append(["No", "No", macro_f1_mlp_mean_vector])

print("MLP usando de entrenamiento train_set")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

# MLP Mean Vector agrandando el conjunto de entrenamiento
res = []

macro_f1_mlp_mean_vector_sin_stop_words_pre_proc_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit, True, True, clf_mlp_mean_vector_sin_stop_words_preproc_agrandado, test_set)
res.append(["Sí","Sí", macro_f1_mlp_mean_vector_sin_stop_words_pre_proc_agrandado])

macro_f1_mlp_mean_vector_sin_stop_words_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_mlp_mean_vector_sin_stop_words_agrandado, test_set)
res.append(["Sí", "No", macro_f1_mlp_mean_vector_sin_stop_words_agrandado])

macro_f1_mlp_mean_vector_pre_proc_agrandado =  macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_mlp_mean_vector_preproc_agrandado, test_set)
res.append(["No", "Sí", macro_f1_mlp_mean_vector_pre_proc_agrandado])

macro_f1_mlp_mean_vector_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_mlp_mean_vector_agrandado, test_set)
res.append(["No", "No", macro_f1_mlp_mean_vector_agrandado])

print("MLP usando de entrenamiento train_set+devel_set (probado en test_set)")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))


MLP usando de entrenamiento train_set
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.563197 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.565241 │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.560286 │
├──────────┼─────────────────┼────────────┤
│ No       │ No              │   0.552427 │
╘══════════╧═════════════════╧════════════╛
MLP usando de entrenamiento train_set+devel_set (probado en test_set)
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.543757 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.541133 │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.55644  │
├──────────┼─────────────────┼────────────┤
│ No       │

In [64]:
# SVM concatenacion de vectores
clf_svm_concatvec = entrenar_svm_concatvec(tamanio,tamanio_vec_concat_we, train_set)
print("F1 Score SVM concatenacion de vectores: ", macro_F1_concatvec_we(tamanio, tamanio_vec_concat_we, clf_svm_concatvec, devel_set))

F1 Score SVM concatenacion de vectores:  0.4121280805187426


In [95]:
# SVM Mean Vector Entrenamiento

clf_svm_mean_vector_sin_stop_words_preproc = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,True,True, train_set)
clf_svm_mean_vector_sin_stop_words = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,True,False, train_set)
clf_svm_mean_vector_preproc = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,False,True, train_set)
clf_svm_mean_vector = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,False,False, train_set)

clf_svm_mean_vector_sin_stop_words_preproc_agrandado = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,True,True, entrenamiento_agrandado)
clf_svm_mean_vector_sin_stop_words_agrandado = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,True,False, entrenamiento_agrandado)
clf_svm_mean_vector_preproc_agrandado = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,False,True, entrenamiento_agrandado)
clf_svm_mean_vector_agrandado = entrenar_svm_mean_vector(cantidad_entradas_vec_tuit,False,False, entrenamiento_agrandado)

In [88]:
# SVM Mean Vector
headers = ['Sin SW', 'Pre-procesado', 'Macro F1']
res = []

macro_F1_svm_mean_vector_sin_stop_words_pre_proc = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,True, clf_svm_mean_vector_sin_stop_words_preproc, devel_set)
res.append(["Sí","Sí", macro_F1_svm_mean_vector_sin_stop_words_pre_proc])

macro_F1_svm_mean_vector_sin_stop_words = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_svm_mean_vector_sin_stop_words, devel_set)
res.append(["Sí","No", macro_F1_svm_mean_vector_sin_stop_words])

macro_F1_svm_mean_vector_sin_preproc = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_svm_mean_vector_preproc, devel_set)
res.append(["No","Sí", macro_F1_svm_mean_vector_sin_preproc])

macro_F1_svm_mean_vector = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_svm_mean_vector, devel_set)
res.append(["No","No", macro_F1_svm_mean_vector])

print("Pruebas con SVM Mean Vector entrenando en train_set")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

# SVM Mean Vector agrandando el conjunto de entrenamiento
res = []

macro_F1_svm_mean_vector_sin_stop_words_pre_proc_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,True, clf_svm_mean_vector_sin_stop_words_preproc_agrandado, test_set)
res.append(["Sí","Sí", macro_F1_svm_mean_vector_sin_stop_words_pre_proc_agrandado])

macro_F1_svm_mean_vector_sin_stop_words_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_svm_mean_vector_sin_stop_words_agrandado, test_set)
res.append(["Sí","No", macro_F1_svm_mean_vector_sin_stop_words_agrandado])

macro_F1_svm_mean_vector_sin_pre_proc_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_svm_mean_vector_preproc_agrandado, test_set)
res.append(["No","Sí", macro_F1_svm_mean_vector_sin_pre_proc_agrandado])

macro_F1_svm_mean_vector_agrandado = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_svm_mean_vector_agrandado, test_set)
res.append(["No","No", macro_F1_svm_mean_vector_agrandado])

print("Pruebas con SVM Mean Vector entrenando en train_set+devel_set (probado en test_set)")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

Pruebas con SVM Mean Vector entrenando en train_set
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.615335 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.607947 │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.589528 │
├──────────┼─────────────────┼────────────┤
│ No       │ No              │   0.588391 │
╘══════════╧═════════════════╧════════════╛
Pruebas con SVM Mean Vector entrenando en train_set+devel_set (probado en test_set)
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.604831 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.60103  │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.610811 │
├──────────┼────────────────

In [66]:
# Ensamblado de votación con SVM y MLP Entrenamiento
clf_ensamblado_votacion_svm_mlp_sin_stop_words_preproc = entrenar_ensamblado_de_votacion_svm_mlp(cantidad_entradas_vec_tuit, True, True, train_set)
clf_ensamblado_votacion_svm_mlp_sin_stop_words = entrenar_ensamblado_de_votacion_svm_mlp(cantidad_entradas_vec_tuit, True, False, train_set)
clf_ensamblado_votacion_svm_mlp_preproc = entrenar_ensamblado_de_votacion_svm_mlp(cantidad_entradas_vec_tuit, False, True, train_set)
clf_ensamblado_votacion_svm_mlp = entrenar_ensamblado_de_votacion_svm_mlp(cantidad_entradas_vec_tuit, False, False, train_set)

In [67]:
# Ensamblado de votación con SVM y MLP
headers = ['Sin SW', 'Pre-procesado', 'Macro F1']
res = []

macro_F1_ensamblado_votacion_svm_mlp_sin_stop_words_preproc = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,True, clf_ensamblado_votacion_svm_mlp_sin_stop_words_preproc, test_set)
res.append(["Sí","Sí", macro_F1_ensamblado_votacion_svm_mlp_sin_stop_words_preproc])

macro_F1_ensamblado_votacion_svm_mlp_sin_stop_words = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_ensamblado_votacion_svm_mlp_sin_stop_words, test_set)
res.append(["Sí","No", macro_F1_ensamblado_votacion_svm_mlp_sin_stop_words])

macro_F1_ensamblado_votacion_svm_mlp_preproc  = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_ensamblado_votacion_svm_mlp_preproc, test_set)
res.append(["No","Sí", macro_F1_ensamblado_votacion_svm_mlp_preproc])

macro_F1_ensamblado_votacion_svm_mlp  = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_ensamblado_votacion_svm_mlp, test_set)
res.append(["No","No", macro_F1_ensamblado_votacion_svm_mlp])

print("Ensamblado de votación entre SVM y MLP")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

Ensamblado de votación entre SVM y MLP
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.585081 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.5879   │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.585909 │
├──────────┼─────────────────┼────────────┤
│ No       │ No              │   0.585845 │
╘══════════╧═════════════════╧════════════╛


### Hiperparámetros

In [82]:
from sklearn.model_selection import GridSearchCV
def encontrar_mejor_hiperparametros(cantidad_entradas_vec, corpus, parametros):
    parametros_busqueda = {
        'svc__C': parametros,
        'svc__kernel': ['linear', 'rbf', 'sigmoid']
    }
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    grid_search = GridSearchCV(clf, parametros_busqueda)
    datos, polaridad = generar_datos_polaridad_mean_vector_we(cantidad_entradas_vec, True, False, corpus)
    grid_search.fit(datos, polaridad)
    return grid_search.best_estimator_

def buscar_mejor_hiperparametros_en_intervalo(cantidad_entradas_vec, corpus, param_1, param_2, limite):
    mejor_param = 0
    mejor_clf = 0
    for i in range(limite):
        print("Buscando mejor paramétro entre: ", param_1, param_2)
        mejor_clf = encontrar_mejor_hiperparametros(cantidad_entradas_vec, corpus, [param_1, param_2])
        mejor_param = mejor_clf[1].C
        print("El mejor clasificador es: ", mejor_clf)

        if mejor_param == param_2:
            peor_param = param_1
        else:
            peor_param = param_2

        if mejor_param > peor_param:
            medio = (mejor_param - peor_param)/2 + peor_param
        else:
            medio = (peor_param - mejor_param)/2 + mejor_param

        param_1 = mejor_param
        param_2 = medio


    return mejor_param, mejor_clf

mejor_c, mejor_modelo = buscar_mejor_hiperparametros_en_intervalo(256, train_set, 1,0.01, 10)

Buscando mejor paramétro entre:  1 0.01
El mejor clasificador es:  Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, gamma='auto'))])
Buscando mejor paramétro entre:  1 0.505
El mejor clasificador es:  Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=1, gamma='auto'))])
Buscando mejor paramétro entre:  1 0.7525
El mejor clasificador es:  Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.7525, gamma='auto'))])
Buscando mejor paramétro entre:  0.7525 0.87625
El mejor clasificador es:  Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.7525, gamma='auto'))])
Buscando mejor paramétro entre:  0.7525 0.814375
El mejor clasificador es:  Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.814375, gamma='auto'))])
Buscando mejor paramétro entre:  0.814375 0.7834375
El mejor clasificador es:  Pipeline(steps=[('standardsca

In [92]:
# Obtener los hiperparámetros utilizados
svm = clf_svm_mean_vector.named_steps['svc']  # Obtener el estimador SVM del pipeline
C = svm.C  # Obtener el valor de C
gamma = svm.gamma  # Obtener el valor de gamma

print(f"El valor de C es {C} y el valor de gamma es {gamma}")

macro_f1_con_mejores_hiperparametros = macro_F1_mean_vector(256,True,True, mejor_modelo, devel_set)
print("Macro F1 probado en devel_set", macro_f1_con_mejores_hiperparametros)

El valor de C es 1.0 y el valor de gamma es auto
Macro F1 probado en devel_set 0.6136053723342164


## Parte 4: Evaluación sobre test

Deben probar los mejores modelos obtenidos en la parte anterior sobre el corpus de test.

También deben comparar sus resultados con un modelo pre-entrenado para análisis de sentimientos de la biblioteca [pysentimiento](https://github.com/pysentimiento/pysentimiento) (deben aplicarlo sobre el corpus de test).



In [42]:
# Pysentimiento
def macro_F1_pysentimiento(corpus):
    analyzer = create_analyzer(task="sentiment", lang="es")
    y_pred = []
    y_real = [elem[2] for elem in corpus]
    for elemento in corpus:
        analizer_result = analyzer.predict(elemento[1]).output
        y_pred.append(sentimiento_map[analizer_result])
    return f1_score(y_real, y_pred, average="macro")

print("F1 Score pysentimiento", macro_F1_pysentimiento(test_set))

F1 Score pysentimiento 0.7041227233971533


In [103]:
# Evaluación sobre test MLP

print("Pruebas MLP sobre test_set")
res = []
headers = ['Sin SW', 'Pre-procesado', 'Macro F1']

macro_f1_mlp_mean_vector_sin_stop_words_pre_proc = macro_F1_mean_vector(cantidad_entradas_vec_tuit, True, True, clf_mlp_mean_vector_sin_stop_words_preproc, test_set)
res.append(["Sí","Sí", macro_f1_mlp_mean_vector_sin_stop_words_pre_proc])

macro_f1_mlp_mean_vector_sin_stop_words = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_mlp_mean_vector_sin_stop_words, test_set)
res.append(["Sí", "No", macro_f1_mlp_mean_vector_sin_stop_words])

macro_f1_mlp_mean_vector_pre_proc =  macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_mlp_mean_vector_preproc, test_set)
res.append(["No", "Sí", macro_f1_mlp_mean_vector_pre_proc])

macro_f1_mlp_mean_vector = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_mlp_mean_vector, test_set)
res.append(["No", "No", macro_f1_mlp_mean_vector])

print(tabulate(res, headers=headers, tablefmt='fancy_grid'))


Pruebas MLP sobre test_set
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.558917 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.56165  │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.556806 │
├──────────┼─────────────────┼────────────┤
│ No       │ No              │   0.551947 │
╘══════════╧═════════════════╧════════════╛


In [104]:
# Prueba SVM Mean Vector sobre test_set
headers = ['Sin SW', 'Pre-procesado', 'Macro F1']
res = []

macro_F1_svm_mean_vector_sin_stop_words_preproc = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,True, clf_svm_mean_vector_sin_stop_words_preproc, test_set)
res.append(["Sí","Sí", macro_F1_svm_mean_vector_sin_stop_words_preproc])

macro_F1_svm_mean_vector_sin_stop_words = macro_F1_mean_vector(cantidad_entradas_vec_tuit,True,False, clf_svm_mean_vector_sin_stop_words, test_set)
res.append(["Sí","No", macro_F1_svm_mean_vector_sin_stop_words])

macro_F1_svm_mean_vector_sin_preproc = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,True, clf_svm_mean_vector_preproc, test_set)
res.append(["No","Sí", macro_F1_svm_mean_vector_sin_preproc])

macro_F1_svm_mean_vector = macro_F1_mean_vector(cantidad_entradas_vec_tuit,False,False,clf_svm_mean_vector, test_set)
res.append(["No","No", macro_F1_svm_mean_vector])

print("Pruebas SVM Mean Vector sobre test_set")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

# SVM Mean Vector agrandando el conjunto de entrenamiento
res = []

res.append(["Sí","Sí", macro_F1_svm_mean_vector_sin_stop_words_pre_proc_agrandado])

res.append(["Sí","No", macro_F1_svm_mean_vector_sin_stop_words_agrandado])

res.append(["No","Sí", macro_F1_svm_mean_vector_sin_pre_proc_agrandado])

res.append(["No","No", macro_F1_svm_mean_vector_agrandado])

print("Pruebas con Mean Vector entrenando en train_set+devel_set")
print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

macro_f1_con_mejores_hiperparametros = macro_F1_mean_vector(256,True,True, mejor_modelo, test_set)
print("Macro F1 probado en test_set con los hiperparámetros mejorados: ",macro_f1_con_mejores_hiperparametros)


Pruebas SVM Mean Vector sobre test_set
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.607559 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.601875 │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.609586 │
├──────────┼─────────────────┼────────────┤
│ No       │ No              │   0.608642 │
╘══════════╧═════════════════╧════════════╛
Pruebas con Mean Vector entrenando en train_set+devel_set
╒══════════╤═════════════════╤════════════╕
│ Sin SW   │ Pre-procesado   │   Macro F1 │
╞══════════╪═════════════════╪════════════╡
│ Sí       │ Sí              │   0.604831 │
├──────────┼─────────────────┼────────────┤
│ Sí       │ No              │   0.60103  │
├──────────┼─────────────────┼────────────┤
│ No       │ Sí              │   0.610811 │
├──────────┼─────────────────┼────────────┤
│ No       │ No        

In [105]:
# Puntuación por clase
headers = ['Conjunto','Exactitud']
res = []

def obtener_tuits_por_clase(corpus):
    positivos = []
    negativos = []
    neutros = []
    for tuit in corpus:
        if tuit[2] == 'P':
            positivos.append(tuit)
        if tuit[2] == 'N':
            negativos.append(tuit)
        if tuit[2] == 'NONE':
            neutros.append(tuit)
    return positivos, negativos, neutros

tuits_positivos, tuits_negativos, tuits_neutros = obtener_tuits_por_clase(test_set)
accuracy_positivos= accuracy_score_mean_vector(cantidad_entradas_vec_tuit,False,True,clf_svm_mean_vector, tuits_positivos)
accuracy_negativos = accuracy_score_mean_vector(cantidad_entradas_vec_tuit,False,True,clf_svm_mean_vector, tuits_negativos)
accuracy_neutros = accuracy_score_mean_vector(cantidad_entradas_vec_tuit,False,True,clf_svm_mean_vector, tuits_neutros)

print("Desglose de la mejor estrategia (SVM Mean Vector) mostrando su comportamiento en cada clase (métrica: exactitud)")
res.append(["Positivos", accuracy_positivos])
res.append(["Negativos", accuracy_negativos])
res.append(["Neutros", accuracy_neutros])


print(tabulate(res, headers=headers, tablefmt='fancy_grid'))

Desglose de la mejor estrategia (SVM Mean Vector) mostrando su comportamiento en cada clase (métrica: exactitud)
╒════════════╤═════════════╕
│ Conjunto   │   Exactitud │
╞════════════╪═════════════╡
│ Positivos  │    0.658842 │
├────────────┼─────────────┤
│ Negativos  │    0.676871 │
├────────────┼─────────────┤
│ Neutros    │    0.497717 │
╘════════════╧═════════════╛


## Preguntas finales

Responda las siguientes preguntas:

### 1) ¿Qué modelos probaron para la representación de los tweets?

Probamos distintas representaciones de Bag of Words (en adelante BoW) y de Word Embeddings (en adelante WE).

Entre las representaciones de BoW aplicamos el Estándar, filtramos stop-stop words, y combinamos con TF-IDF que mide la relevancia de una palabra con respecto al texto que lo contiene, para luego asignarle un factor de ponderación.

Con respecto a WE utilizamos dos enfoques diferentes: el primero es obteniendo la mediana del valor de las palabras que conforman un tweet (Mean Vector). El segundo es concatenación de vectores, en el que se genera una lista con la concatenación de los valores de las palabras de los tweets (Concat Vec).

### 2) ¿Aplicaron algún tipo de preprocesamiento de los textos?

En varios de los experimentos aplicamos prerpocesamiento, entre estas tareas se incluye: reemplazo de identificadores de usuario y Hashtags por tags genéricos y mejora de ortografía. Por lo general, la diferencia entre trabajar con un corpus preprocesado y no preprocesado representa una diferencia en macro F1 de menos de 1%.

A modo de ejemplo: en SVM Mean Vector sin preprocesamiento el macro F1 dió 0,5919, mientras que la misma técnica con prepoc dió 0,5875. En este caso el preprocesamiento empeoró el resultado.

Sin embargo, si eliminamos las Stop-Words y aplicamos esa misma técnica: vemos que sin preproc el resultado es 0,5958, mientras que con un corpus prepocesado el resultado es 0,6001. Por lo que en este caso, la etapa de preprocesado mejoró la medida.

### 3) ¿Qué modelos de aprendizaje automático probaron?

En el desarrollo del laboratorio se generaron distantas tablas mostrando la performance de las distintas estrategias utilizadas con su respectiva representación. La información disponible permite hacer comparaciones directas entre ellas, por lo que en esta sección serán referenciadas extensivamente.

Las estrategias utilizadas fueron:

*   Multi Layer Perceptron (MLP)
Las redes neuronales MLP están compuesta por múltiples capas de neuronas interconectadas.

Este enfoque no tuvo el mejor desempeño en esta tarea, llegando a una puntuación F1 de 0,56. Por más de que no fue el enfoque en el cuál se obtuvieron los mejores resultados en devel, decidimos probarlo con el corpus de prueba para ver si habían cambios sustantivos en el resultado, sin mucho éxito.

Nuestros experimentos con MLP fueron realizados en ambas representaciones (BoW y WE) y

*   Support Vector Machines (SVM):

En forma simplifacada, en este modelo los tweets son reperesentados como puntos en el espacio. En el entrenamiento se busca encontrar un hiperplano que separe los tweets de acuerdo a la clasificación dada.

Experimentamos con ambas versiones de WE de los tweets (MV y concat). Implementamos una búsqueda automática de hiperparámetros, pero los resultados de esta no fueron suficientes para superar los parámetros por defecto (C=1, kernel="linear"). La forma que se realizó la búsqueda fue utilizando un método de bipartición, se sospecha que la función que relaciona a los HP con el score parece no ser continua, pero no hay evidencia para afirmarlo.

Los resultados de los experimentos fueron evaluados por el grupo de forma positiva, llegando a

*   LSTM:

Este enfoque será detallado en la pregunta 5.

### 4) ¿Qué atributos utilizaron para estos modelos?

Para las tres estrategias definimos dos _«parámetros»_ vinculados con el prepocesamiento y la eliminación de stop-words en el corpus. En las tablas se puede ver las distintas combinaciones y sus resultados, vimos que no hay una correlación clara entre una puntuación superior con alguna de las combinaciones en particular. Además, vemos que la puntuación varía en un margen muy pequeño cuando se varían estos parámetros (no más de 1%).

Por otro lado, para SVM implementamos una búsqueda automática de hiperparámetros, pero los resultados de esta no fueron suficientes para superar los parámetros por defecto (C=1, kernel="linear"). La forma que se realizó la búsqueda fue utilizando un método de bipartición, se sospecha que la función que relaciona a los HP con el score no es continua, pero no hay evidencia para afirmarlo. Es importante tener en cuenta que las pruebas sobre los hiperparametros se hicieron con la representación mean vector con vectores de tamaño 256 en vez de 300 (como se realizó en el resto de las pruebas), ya que un vector tan grande hizo que demorara un tiempo excesivo la búsqueda de parámetros.

Por último, buscamos parámetros utilizando heurísticas para mejorar los resultados, sin obtener éxito en los casos de SVM y MLP. Los parámetros por default fueron los que tuvieron mejores resultados en todos los casos.

Para LSTM realizamos tanto búsqueda automática como búsqueda manual, esto será detallado en la pregunta 6.

### 5) ¿Probaron algún enfoque de aprendizaje profundo?

Sí, trabajamos con Long Short-Term Memory (LSTM).
Con fines de prolijidad, en el Laboratorio solamente dejamos la configuración que dio mejores resultados. Se generó una red con varias capas densas, se realizaron varias pruebas variando el tamaño de las capas para poder elegir la que funcione mejor. Después de las pruebas la red resultante fue bastante simple y no mostró un desempeño destacado entre los otros modelos, fue el que funcionó peor. La máxima puntuación F1 lograda fue 0.324333, apenas pasando la mitad de la medida alcanzada por la mejor estrategia.

### 6) ¿Probaron diferentes configuraciones de hiperparámetros?

Las pruebas en LSTM se realizaron variando el tamaño de las capas, cambiando el tamaño de los batches en los que se fue aprendiendo el conjunto de entrenamiento y la cantidad de épocas. Otra prueba realizada que no mostró una mejoría en el resultado final fue agregando capas de "Dropout" donde solo un porcentaje de las neuronas llegaran activas a la siguiente capa, esto se hace para reducir el sobre ajuste.
Las pruebas cambiando la cantidad de neuronas de una capa se realizaron a través de un paramétro de entrada en la función que genera el modelo, ese parametro se colocó en lugar de la cantidad de neuronas de la capa y se generaron varios modelos calculando para cada uno su puntaje macro F1, para así poder elegir el que funcione mejor. De la misma forma se ejecutaron las pruebas con las capas dropout.
Otro parámetro que fue modificado fue el tamaño del vector de entrada, se observó que un vector de tamaño 256 fue el que dio mejor resultado.

### 7) ¿Qué enfoque (preprocesamiento + representación de tweets + modelo + atributos/parámetros) obtuvo la mejor Macro-F1?

Representación de Tweets: WE - Mean Vector

Embedding utilizado: embeddings-l-model.vec
Modelo: SVM

Atributos/Parámetros:

Sin stop-words: False
Corpus preprocesado: True
Macro F1 = 0.609586

Nota importante: Uno de nuestros experimientos se basó en fusionar el devel_set con el train_set para generar un corpus de entrenamiento más grande para ver si esto influía en el resultado final. Con este nuevo corpus utilizamos la estrategia de SVM Mean Vector y MLP. El resultado fue un incremento llegando al recórd del grupo con 0.610811. La diferencia entre este enfoque y su contraparte solo entrenada con el train_set es de 0.001225. Entendemos que la idea de la tarea es utilizar correctamente las tres particiones del corpus, sin embargo nos pareció interesante mostrar este resultado destacando que aumentando el tamaño del corpus los resultados mejoraron (muy poco).

Los datos específicos de este experimento son:

Representación de Tweet: WE - Mean Vector

Modelo: SVM

Corpus de entrenamiento: train_set + devel_set

Embedding utilizado: embeddings-l-model.vec
Atributos/Parámetros:

Sin stop-words = False
Corpus preprocesado = True
Macro F1 = 0.610811

### 8) ¿Qué clase es la mejor clasificada por este enfoque? ¿Cuál es la peor? ¿Por qué piensan que sucede esto?

Utilizamos la Exactitud (predicciones correctas / predicciones totales) sobre cada clase separada para obtener métricas sobre la performance de cada una. Los resultados se pueden observar en la tabla correspondiente.

La clase mejor clasificada fueron los Tweets Negativos con 0.67871 y muy cerca los Tweets Positivos con 0,658842. Por otro lado, la peor clasificada fueron los Tweets Neutros, con una exactitud de 0.49717.

Creemos que esto sucede por las siguientes razones:
- A diferencia de los tweets positivos o negativos, que pueden contener palabras fuertemente emocionales que «muevan la aguja» para un polo u otro, los tweets neutrales pueden carecer de señales claras de polaridad.
- La polaridad de un tweet puede depender del contexto en el que se utilice. Una misma frase puede interpretarse como neutra en un contexto, pero como positiva o negativa en otro.

### 9) ¿Cómo son sus resultados en comparación con los de pysentimiento? ¿Por qué piensan que sucede esto?

Nuestros resultados son inferiores a los de pysentimiento por un márgen aproximado de 10% a 15% dependiendo del enfoque, creemos que esto se debe a las siguientes razones:

1. Nuestros modelos son entrenados en base a tweets: Como fue identificado en la tarea anterior, los tweets tienen dos grandes problemas: El primero de ellos se debe a las faltas ortografía y el uso indebido o informal del lenguaje. Esto lleva a que estrategias como BoW Estándar y BoW TF-IDF pierdan efectividad y eficacia, ya que al tener una misma palabra escrita de varias formas se pierde la escencia del multiconjunto.

   Este problema afecta también a la estrategia WE, ya que el embedding utilizado no solo sucede que las palabras mal escritas no están en el embedding, sino que hay algunas pablaras correctas que simplemente no están representadas en el embedding. Esto, en definitiva, es equivalente afirmar que aquellas palabras no representadas en el embedding es como si no estuviesen estén en el tweet.

  Por otro lado: el segundo problema de los tweets son su longitud y consecuente falta de contexto. Los tweets están limitados a 280 caracteres, lo que hace que la cantidad de información disponible para el entrenamiento sea limitada.

2. A pesar de estar entrenados con el mismo corpus (TASS 2020) la librería pysentimiento utiliza una variante de BERT (RoBERTuito). Por lo que entendimos de la documentación, RoBERTuito es un modelo pre-entrenado en más de 500 millones de tweets, esto marca una ventaja cuantitativa en la etapa de entrenamiento para pysentimiento que explicaría la diferencia en la medida F1.

  Como fue mencionado en la respuesta 7: uno de nuestros experimientos se basó en fusionar el devel_set con el train_set para generar un corpus de entrenamiento más grande para ver si esto influía en el resultado final. El resultado fue un incremento de 2 puntos llegando al recórd del grupo con 61,08%. Este resultado es conclusivo y reafirma algo evidente: el tamañano del corpus es relevante, por lo que pre-entrenar sobre 500 millones de tweets marca una ventaja sustantiva.

  BERT despertó nuestra curiosidad para experimentar con él, pero dado que viene pre-entrenado, entendimos que no es la esencia de esta tarea trabajar con este tipo de modelo de redes neuronales.

3. Al igual que el implementado por nosotros, Pysentimiento incluye un preprocesador de texto que reemplaza los identificadores de usuario y las URL por tokens especiales. Sin embargo, tiene otras funcionalidades como por ejemplo acortar caracteres repetidos y separar hasthags por palabras correctamente. El método _preprocess_tweet()_ de pysentimiento cuenta con 11 parámetros para configurar el tipo de preprocesado. Esto muestra que el preprocesador de la librería es mucho más sofisticado que el nuestro.